In [1]:
import awkward as ak
import numpy as np
import uproot as uproot
import matplotlib.pyplot as plt
import mplhep as hep
import vector as vec
from tqdm import tqdm
import ROOT
import math
import re
'''
from utils import *
from utilsPCA import *
from plotting import *
from efficiency import *
from isolation import *
'''
import matplotlib as mpl

#mpl.rc('xtick', labelsize=16) 
#mpl.rc('ytick', labelsize=16) 
#mpl.rc('axes', labelsize=16, titlesize=20)
#mpl.rcParams["legend.title_fontsize"]=18

#plt.style.use(hep.style.CMS)
%matplotlib inline
lw=1.2

ROOT.enableJSVis()
    
import pickle



folders_1p9 = ["SinglePionTiming_1p9_100GeV", "SinglePionTiming_1p9_10GeV",
 "SinglePionTiming_1p9_15GeV", "SinglePionTiming_1p9_2GeV",
 "SinglePionTiming_1p9_30GeV", "SinglePionTiming_1p9_4GeV",
 "SinglePionTiming_1p9_50GeV", "SinglePionTiming_1p9_6GeV",
 "SinglePionTiming_1p9_8GeV"]


folders_2p2 = ["SinglePionTiming_2p2_100GeV", "SinglePionTiming_2p2_10GeV",
 "SinglePionTiming_2p2_15GeV", "SinglePionTiming_2p2_2GeV",
 "SinglePionTiming_2p2_30GeV", "SinglePionTiming_2p2_4GeV",
 "SinglePionTiming_2p2_50GeV", "SinglePionTiming_2p2_6GeV",
 "SinglePionTiming_2p2_8GeV"]



Welcome to JupyROOT 6.30/04


In [10]:

def compute_residuals_energiesALL(file_name):
    
    open_file = uproot.open(file_name)
    f_simTICLCandidate = open_file["ticlDumper/simTICLCandidate;1"]
    f_candidates = open_file["ticlDumper/candidates;1"]
    f_trackstersTICL=open_file["ticlDumper/trackstersTiclCandidate;1"]
    f_associations = open_file["ticlDumper/associations;1"]
    candidate_time = f_candidates["candidate_time"].array()
    candidate_timeErr= f_candidates["candidate_timeErr"].array()
    
    candidate_time_MTD = f_candidates["candidate_time_MTD"].array()
    candidate_time_MTDErr= f_candidates["candidate_time_MTD_err"].array()
    
    candidate_energy= f_candidates["candidate_energy"].array()
    candidate_px= f_candidates["candidate_px"].array()
    candidate_py= f_candidates["candidate_py"].array()
    trackster_energy=f_trackstersTICL["raw_energy"].array()#f_candidates["candidate_raw_energy"].array()
    
    print(f_associations["ticlCandidate_simToReco_CP"].array()[0])
    print(len(f_associations["ticlCandidate_simToReco_CP"].array()[0]))
    assoc=f_associations["ticlCandidate_simToReco_CP"].array()
    assoc_shE=f_associations["ticlCandidate_simToReco_CP_sharedE"].array()

    #print(len(assoc[1]),len(candidate_energy[1]))

    #print(candidate_energy[0][6])
    x=0
    #print(sum([len(y) for y in candidate_energy]))
    ncandidates=sum([len(y) for y in candidate_energy])
    counter=0
    listvalid=[]
    
    for i in tqdm(range(len(assoc))):
        listvalid.append([])
        for j in range(len(assoc[i])):
            for k in range(len(assoc[i][j])):
                #print(i,j,k)
                #if assoc[i][j][k]==3 and i==118:
                #    print("--" , trackster_energy[i][assoc[i][j][k]], "sh:",assoc_shE[i][j][k])
                
                if assoc_shE[i][j][k]>trackster_energy[i][assoc[i][j][k]]*0.5:
                    listvalid[i].append(assoc[i][j][k])
                    if assoc_shE[i][j][k]/trackster_energy[i][assoc[i][j][k]]>1.001:
                        print("sus", i,j,k,assoc[i][j][k])
                    #print("top")
                    #print(assoc_shE[i][j][k],candidate_energy[i][assoc[i][j][k]])
                    counter+=1
                #candidate_energy[i][j][assoc[i][j][k]]

    print("efficient:", counter,"inefficient linking or trackless:", ncandidates-counter,"efficiency:",counter/ncandidates)
    print("trackless candidates:")
    
    #print("")
    
    
    tracksters_in_candidate=f_candidates["tracksters_in_candidate"].array()
    
    
    simcandidate_time = f_simTICLCandidate["simTICLCandidate_time"].array()
    simcandidate_charge=f_simTICLCandidate["simTICLCandidate_charge"].array()
    
    candidate_charge=f_candidates["candidate_charge"].array()
    
    ticl_residuals={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    ticl_energies={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    ticl_pt={"avg":[],
             "hgcal":[],
             "mtd":[]}
    
    ticl_isCharged={"avg":[],
                    "hgcal":[],
                    "mtd":[]}
    trackless=0
    for i in tqdm(range(int(len(simcandidate_time)))):
        #if ak.max(assoc[i])>len(candidate_time[i]):
        #if len(candidate_time[i])>len(trackster_energy[i]):
        #        print(i, len(candidate_time[i]),len(candidate_energy[i]),len(trackster_energy[i]),ak.max(assoc[i]))
        #        print(trackster_energy[i])
        for j in range(len(candidate_time[i])):
            if (len(tracksters_in_candidate[i][j])>1):
                print("ATTENZIONE")
            t_MTD=False
            t_HGCal=False
            time_avg=0
            time_avgErr=0
            trck_in_cand=tracksters_in_candidate[i][j]
            if (len(trck_in_cand)>0):
                if(len(trck_in_cand)>1): 
                    print("---- Warning: more than 1 trackster in candidate")
                if trck_in_cand[0] in listvalid[i]:
                    if (candidate_timeErr[i][j]>0):
                        t_HGCal=True 
                        ticl_residuals["hgcal"].append(simcandidate_time[i][0]-candidate_time[i][j])
                        ticl_energies["hgcal"].append(candidate_energy[i][j])
                        ticl_pt["hgcal"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["hgcal"].append(not candidate_charge[i][j]==0)
                        time_avg=candidate_time[i][j]
                        time_avgErr=candidate_timeErr[i][j]

                    if(candidate_time_MTDErr[i][j]>0):
                        t_MTD=True
                        MTD_time=candidate_time_MTD[i][j]
                        ticl_residuals["mtd"].append(simcandidate_time[i][0]-MTD_time)
                        ticl_energies["mtd"].append(candidate_energy[i][j])
                        ticl_pt["mtd"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["mtd"].append(not candidate_charge[i][j]==0)
                        if(t_HGCal):
                            inv_hgcal_err=1./(time_avgErr**2)
                            inv_mtd_err=1./(candidate_time_MTDErr[i][j]**2)
                            time_avg=(candidate_time[i][j]*inv_hgcal_err+MTD_time*inv_mtd_err)/(inv_mtd_err+inv_hgcal_err)

                        else:
                            time_avg=MTD_time

                    if(t_MTD or t_HGCal):
                        ticl_residuals["avg"].append(simcandidate_time[i][0]-time_avg)
                        ticl_energies["avg"].append(candidate_energy[i][j])
                        ticl_pt["avg"].append(math.sqrt(candidate_px[i][j]**2+candidate_py[i][j]**2))
                        ticl_isCharged["avg"].append(not candidate_charge[i][j]==0)
            else:
                trackless+=1
            
            eff_track=(ncandidates-trackless)/ncandidates
            eff_link = counter/(ncandidates-trackless)
                        
    return ticl_residuals, ticl_energies, ticl_pt, ticl_isCharged,eff_track,eff_link


def eff_calculator(neff,ineff):
    k=neff
    n=ineff
    return math.sqrt((k+1)*(k+2)/((n+2)*(n+3))-(k+1)**2/((n+2)**2))


def err_eff(neff,tot):
    k=neff
    n=tot
    return math.sqrt((k+1)*(k+2)/((n+2)*(n+3))-(k+1)**2/((n+2)**2))

In [11]:
def pdf(x, par):
    A, mean, sigmaL, alphaL, sigmaR, alphaR = par

    x = np.asarray(x)
    if x.ndim == 0:
        x = np.array([x])

    answer = np.zeros(x.shape)

    left = x < mean
    right = np.logical_not(left)

    top = np.square(x - mean)

    bottomL = 2 * (sigmaL * sigmaL + alphaL * top[left])
    bottomR = 2 * (sigmaR * sigmaR + alphaR * top[right])

    answer[left] = A * np.exp(-np.divide(top[left], bottomL))
    answer[right] = A * np.exp(-np.divide(top[right], bottomR))
    return answer


def cruijff(x, params):
    A = params[0]
    m = params[1]
    sigma_left = params[2]
    sigma_right = params[3]
    alpha_left = params[4]
    alpha_right = params[5]
    
    t = (x[0] - m) 
    if t < 0:
        t = (x[0] - m)/sigma_left 
        return A*math.exp(-0.5 * t * t / (1 + 0.5*alpha_left * t * t))
    else:
        t = (x[0] - m)/sigma_right
        return A*math.exp(-0.5 * t * t / (1 + 0.5*alpha_right * t * t))
    
def res_E_fit(x,par):
    S=par[0]
    N=par[1]
    C=par[2]
    
    return math.sqrt( (S/math.sqrt(x[0]))**2+(N/x[0])**2+C**2)

    

# Funz per fittare

# Load selected data >

In [15]:
file = open('data_assoc/2p2.pickle', 'rb')
dict_data = pickle.load(file)
file.close()

print(dict_data[folders_2p2[2]][4])

        

eff_calo=ROOT.TGraphErrors()
eff_calotrack=ROOT.TGraphErrors()
eff_PID=ROOT.TGraphErrors()
counter=0

numeri_GeV=[]
for folder in folders_1p9:
    match = re.search(r'(\d+)GeV', folder)
    if match:
        numeri_GeV.append(int(match.group(1)))
        eff_calo.AddPoint(int(match.group(1)),dict_data[folder][4]/dict_data[folder][7])
        eff_calo.SetPointError(counter,0,err_eff(dict_data[folder][4],dict_data[folder][7]))
        eff_calotrack.AddPoint(int(match.group(1)),dict_data[folder][5]/dict_data[folder][7])
        eff_calotrack.SetPointError(counter,0,err_eff(dict_data[folder][5],dict_data[folder][7]))
        eff_PID.AddPoint(int(match.group(1)),dict_data[folder][6]/dict_data[folder][7])
        eff_PID.SetPointError(counter,0,err_eff(dict_data[folder][6],dict_data[folder][7]))
        
        counter+=1
        

        
mg_eff=ROOT.TMultiGraph()

eff_calo.SetLineColor(ROOT.kRed+1)
eff_calo.SetMarkerColor(ROOT.kRed+1)
eff_calo.SetMarkerStyle(20)
eff_calo.SetTitle("Calorimetric efficiency;E [GeV];Efficiency")


eff_calotrack.SetLineColor(ROOT.kSpring-7)
eff_calotrack.SetMarkerColor(ROOT.kSpring-7)
eff_calotrack.SetMarkerStyle(22)
eff_calotrack.SetTitle("CaloTrack efficiency;E [GeV];Efficiency")


eff_PID.SetLineColor(ROOT.kViolet+4)
eff_PID.SetMarkerColor(ROOT.kViolet+4)
eff_PID.SetMarkerStyle(23)
eff_PID.SetTitle("PID efficiency;E [GeV];Efficiency")


ceff=ROOT.TCanvas()
ceff.cd()
mg_eff.SetTitle("Efficiencies for #eta=1.9;p_{T}[GeV/c];Efficiency")
mg_eff.Add(eff_calo)
mg_eff.Add(eff_calotrack)
mg_eff.Add(eff_PID)
mg_eff.Draw("AP")
ceff.BuildLegend()
mg_eff.SetMinimum(0.)
mg_eff.SetMaximum(1.)

ceff.SaveAs("PlotAssoc/eff.pdf")







35412


Info in <TCanvas::Print>: pdf file PlotAssoc/eff.pdf has been created


In [16]:
file = open('data_assoc/2p2.pickle', 'rb')
dict_data = pickle.load(file)
file.close()

print(dict_data[folders_2p2[2]][4])

        

eff_calo=ROOT.TGraphErrors()
eff_calotrack=ROOT.TGraphErrors()
eff_PID=ROOT.TGraphErrors()
counter=0

numeri_GeV=[]
for folder in folders_2p2:
    match = re.search(r'(\d+)GeV', folder)
    if match:
        numeri_GeV.append(int(match.group(1)))
        eff_calo.AddPoint(int(match.group(1)),dict_data[folder][4]/dict_data[folder][7])
        eff_calo.SetPointError(counter,0,err_eff(dict_data[folder][4],dict_data[folder][7]))
        eff_calotrack.AddPoint(int(match.group(1)),dict_data[folder][5]/dict_data[folder][7])
        eff_calotrack.SetPointError(counter,0,err_eff(dict_data[folder][5],dict_data[folder][7]))
        eff_PID.AddPoint(int(match.group(1)),dict_data[folder][6]/dict_data[folder][7])
        eff_PID.SetPointError(counter,0,err_eff(dict_data[folder][6],dict_data[folder][7]))
        
        counter+=1
        

        
mg_eff=ROOT.TMultiGraph()

eff_calo.SetLineColor(ROOT.kRed+1)
eff_calo.SetMarkerColor(ROOT.kRed+1)
eff_calo.SetMarkerStyle(20)
eff_calo.SetTitle("Calorimetric efficiency;E [GeV];Efficiency")


eff_calotrack.SetLineColor(ROOT.kSpring-7)
eff_calotrack.SetMarkerColor(ROOT.kSpring-7)
eff_calotrack.SetMarkerStyle(22)
eff_calotrack.SetTitle("CaloTrack efficiency;E [GeV];Efficiency")


eff_PID.SetLineColor(ROOT.kViolet+4)
eff_PID.SetMarkerColor(ROOT.kViolet+4)
eff_PID.SetMarkerStyle(23)
eff_PID.SetTitle("PID efficiency;E [GeV];Efficiency")


ceff=ROOT.TCanvas()
ceff.cd()
mg_eff.SetTitle("Efficiencies for #eta=2.2;p_{T}[GeV/c];Efficiency")
mg_eff.Add(eff_calo)
mg_eff.Add(eff_calotrack)
mg_eff.Add(eff_PID)
mg_eff.Draw("AP")
ceff.BuildLegend()
mg_eff.SetMinimum(0.)
mg_eff.SetMaximum(1.)

ceff.SaveAs("PlotAssoc/eff_2p2.pdf")



35412


Info in <TCanvas::Print>: pdf file PlotAssoc/eff_2p2.pdf has been created


In [9]:


from multiprocessing import Process, Queue




def resolution(folder,tag,dict_resolutions,Q):
    
    
    t_res_all, energies_all, pt_all, isCharged_all,eff_link,eff_track=dict_data[folder]
   
    nbins=100
    


    histo={"avg":ROOT.TH1F("combined","Res Combination;Res [ns];Counts",nbins,-0.1,0.1),
          "hgcal":ROOT.TH1F("hgcal","Res HGCal;Res [ns];Counts",nbins,-0.1,0.1),
          "mtd":ROOT.TH1F("mtd","Res MTD;Res [ns];Counts",nbins,-0.1,0.1)}
    
    #f_gaus=ROOT.TF1("f1","gaus(0)",-10.,10.)
    f_cruijff=ROOT.TF1("f2",cruijff,-10,10.,6)

    for key in t_res_all:
        for i in range(len(t_res_all[key])):
            if isCharged_all[key][i]:
                histo[key].Fill(t_res_all[key][i])



    ROOT.gStyle.SetOptFit(True)

    #mg=ROOT.TMultiGraph()

    c1=ROOT.TCanvas()
    c1.cd()
    histo["avg"].SetLineColor(ROOT.kMagenta+2)

    histo["hgcal"].SetLineColor(ROOT.kCyan+2)


    histo["mtd"].SetLineColor(ROOT.kSpring+4)

    [histo[key].SetLineWidth(2) for key in histo]
    [histo[key].Sumw2() for key in histo]
    [histo[key].Draw("SAME E") for key in histo]
    
    c1.BuildLegend()
    
    f_cruijff.SetParNames("A","#mu","#sigma_L","#sigma_R","#alpha_L","#alpha_R")
    f_cruijff.SetParameters(3000,0.0019,0.0173,0.0173,0,0)
    f_cruijff.SetParLimits(2,0.01,10)
    f_cruijff.SetParLimits(3,0.01,10)
    f_cruijff.SetParLimits(4,0.00001,10)
    f_cruijff.SetParLimits(5,0.00001,10)
    
    match = re.search(r'(\d+)GeV', folder)
    
    
    
    
    
    resolutions={}
    resolution_errs={}
    
    for key in t_res_all:
        if(int(match.group(1))==2):
            print(2)
            f_cruijff.SetParameters(200,-0.0007,0.0224223,0.0224223,0.01,0.01)
            f_cruijff.FixParameter(4,0.0001)
            f_cruijff.FixParameter(5,0.0001)
            fitresult=histo[key].Fit(f_cruijff,"LN","",-0.03,0.03)
            f_cruijff.SetParLimits(2,0.01,10)
            f_cruijff.SetParLimits(3,0.01,10)
            f_cruijff.ReleaseParameter(4)
            f_cruijff.ReleaseParameter(5)
            #f_cruijff.FixParameter(4,0.0001)
            #f_cruijff.FixParameter(5,0.0001)
        if(int(match.group(1))==4):
            print(2)
            f_cruijff.SetParameters(200,-0.0007,0.0224223,0.0224223,0.01,0.01)
            f_cruijff.FixParameter(4,0.0001)
            f_cruijff.FixParameter(5,0.0001)
            fitresult=histo[key].Fit(f_cruijff,"LN","",-0.03,0.03)
            f_cruijff.SetParLimits(2,0.01,10)
            f_cruijff.SetParLimits(3,0.01,10)
            f_cruijff.ReleaseParameter(4)
            f_cruijff.ReleaseParameter(5)
            #f_cruijff.FixParameter(4,0.0001)
            #f_cruijff.FixParameter(5,0.0001)
        
        print(folder)
        f_cruijff.ReleaseParameter(4)
        f_cruijff.ReleaseParameter(5)
        fitresult=histo[key].Fit(f_cruijff,"MLS+","SAME",-0.1,0.1)
        cov_matrix=fitresult.GetCovarianceMatrix()
        params=f_cruijff.GetParameters()
        sigma_c=(params[2]+params[3])/2

        resolutions[key]=sigma_c
        resolution_errs[key]=math.sqrt(0.25*(cov_matrix[2][2]+cov_matrix[3][3])+ 0.5*cov_matrix[2][3])
    
    match = re.search(r'(\d+)GeV', folder)
    
    dict_resolutions[int(match.group(1))]=[resolutions,resolution_errs]
    Q.put(dict_resolutions)
    if "1p9" in folder:
        c1.SaveAs("PlotAssoc/H"+str(tag)+".pdf")
        
    if "2p2" in folder:
        c1.SaveAs("PlotAssoc/H"+str(tag)+".pdf")
        
    c1.SaveAs("H"+str(tag)+".pdf")
    

# Fit for 1.9

In [ ]:
procs=[]

q=Queue()
dict_resolutions={}
res={}

    
counter=0
for enkey in dict_resolutions.keys():
    for typekey in dict_resolutions[enkey][0]:
        g_dict[typekey].AddPoint(enkey*math.cosh(1.9),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        g_dict[typekey].SetPointError(counter,0,dict_resolutions[enkey][1][typekey])
    counter=counter+1
cmg=ROOT.TCanvas()
cmg.cd()
mg=ROOT.TMultiGraph()



    

    
for job in procs:
    job.join()

for i in range(len(folders_1p9)):
    dict_resolutions.update(q.get())
    
print(dict_resolutions)

    
    
import pickle


with open('data_assoc/res1p9.pickle', 'wb') as handle:
    pickle.dump(dict_resolutions, handle, protocol=pickle.HIGHEST_PROTOCOL)



# Multiplot for 1.9

In [7]:


file = open('data_assoc/res1p9.pickle', 'rb')
dict_resolutions = pickle.load(file)
file.close()

print(dict_resolutions)



g_dict={"avg":ROOT.TGraphErrors(),
        "hgcal":ROOT.TGraphErrors(),
        "mtd":ROOT.TGraphErrors()}

print(dict_resolutions)
print(dict_resolutions[2])
counter=0
for enkey in dict_resolutions.keys():
    for typekey in dict_resolutions[enkey][0]:
        g_dict[typekey].AddPoint(enkey*math.cosh(1.9),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        g_dict[typekey].SetPointError(counter,0,dict_resolutions[enkey][1][typekey])
    counter=counter+1
cmg=ROOT.TCanvas()
cmg.cd()
mg=ROOT.TMultiGraph()


g_dict["avg"].SetLineColor(ROOT.kMagenta+2)
g_dict["avg"].SetMarkerColor(ROOT.kMagenta+2)
g_dict["avg"].SetMarkerStyle(22)
g_dict["avg"].SetTitle("Combination")

g_dict["hgcal"].SetLineColor(ROOT.kCyan+2)
g_dict["hgcal"].SetMarkerColor(ROOT.kCyan+2)
g_dict["hgcal"].SetMarkerStyle(20)
g_dict["hgcal"].SetTitle("HGCal time")


g_dict["mtd"].SetLineColor(ROOT.kSpring+4)
g_dict["mtd"].SetMarkerColor(ROOT.kSpring+4)
g_dict["mtd"].SetMarkerStyle(34)
g_dict["mtd"].SetTitle("MTD time")
#[g_dict[x].SetTitle(str(x)) for x in g_dict.keys()]

#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]


f_res_E=ROOT.TF1("fres",res_E_fit,0.001,600.,3)#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]
f_res_E.SetNpx(1000)
f_res_E.SetParameters(1.,1.,0.016)
f_res_E.SetParNames("S","N","C")


ROOT.gStyle.SetOptFit(True)

g_dict["hgcal"].Fit(f_res_E,"r")

[mg.Add(g_dict[x]) for x in g_dict.keys()]

mg.SetTitle("Time resolution;E [GeV]; Resolution [ns]")
mg.Draw("AP")

#cmg.BuildLegend()
legend=ROOT.TLegend(0.2,0.8,0.4,0.7)

legend.AddEntry(g_dict["avg"])
legend.AddEntry(g_dict["mtd"])
legend.AddEntry(g_dict["hgcal"])
legend.Draw()

text = ROOT.TLatex()
text.SetNDC(1)  # Imposta le coordinate in termini di frazione del canvas (0-1)
text.SetTextSize(0.04)  # Imposta la dimensione del testo
text.SetTextAlign(31)  # Allinea il testo a destra rispetto alla posizione specificata
text.DrawLatex(0.9, 0.92, "#eta=1.9")  # (x, y, testo) posizione nell'angolo in alto a destra
#mg.SetMinimum(0.)
#mg.SetMaximum(0.035)



mg.SetMinimum(0.)
mg.SetMaximum(0.040)

cmg.Draw()

cmg.SaveAs("PlotAssoc/Resolution_1p9_E_fit.pdf")





{100: [{'avg': 0.01311780291647428, 'hgcal': 0.015326940622618565, 'mtd': 0.024932838659425766}, {'avg': 8.889552678055671e-05, 'hgcal': 0.00011000973457302904, 'mtd': 0.00022405925518237426}], 10: [{'avg': 0.01601317541391399, 'hgcal': 0.01985877867769672, 'mtd': 0.024465657099086588}, {'avg': 0.0001226341549040582, 'hgcal': 0.0001720952569699049, 'mtd': 0.00022924457422755654}], 15: [{'avg': 0.015057893167796795, 'hgcal': 0.018748227167943596, 'mtd': 0.024273191728692457}, {'avg': 0.00010968190214033671, 'hgcal': 0.00015130737734392804, 'mtd': 0.000219105556746429}], 2: [{'avg': 0.02160226973848536, 'hgcal': 0.030221748144080136, 'mtd': 0.02416055121253072}, {'avg': 0.0003893708428934099, 'hgcal': 0.0009673531052199155, 'mtd': 0.000430797660659114}], 30: [{'avg': 0.013966749346122517, 'hgcal': 0.016830783372717557, 'mtd': 0.024522016138515478}, {'avg': 9.410127173520905e-05, 'hgcal': 0.00012264474051159262, 'mtd': 0.00021786194093759002}], 4: [{'avg': 0.01789877574000117, 'hgcal': 0.

Info in <TCanvas::Print>: pdf file PlotAssoc/Resolution_1p9_E_fit.pdf has been created


# Fit 2p2

In [8]:
procs=[]

q=Queue()
dict_resolutions={}
res={}
    
numeri_GeV=[]

for folder in folders_2p2:
    match = re.search(r'(\d+)GeV', folder)
    if match:
        numeri_GeV.append(int(match.group(1)))
    p=Process(target=resolution,args=(folder,folder,dict_resolutions,q))
    procs.append(p)
    p.start()
    p.join()
    #t_resolution,t_resolution_err=resolution(folder,folder,dict_resolutions)



    

    
for job in procs:
    job.join()

for i in range(len(folders_1p9)):
    dict_resolutions.update(q.get())
    
print(dict_resolutions)

    
    
import pickle


with open('data_assoc/res2p2.pickle', 'wb') as handle:
    pickle.dump(dict_resolutions, handle, protocol=pickle.HIGHEST_PROTOCOL)



SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_100GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_10GeV
SinglePionTiming_2p2_15GeV
SinglePionTiming_2p2_15GeV
SinglePionTiming_2p2_15GeV
2
SinglePionTiming_2p2_2GeV
2
SinglePionTiming_2p2_2GeV
2
SinglePionTiming_2p2_2GeV
SinglePionTiming_2p2_30GeV
SinglePionTiming_2p2_30GeV
SinglePionTiming_2p2_30GeV
2
SinglePionTiming_2p2_4GeV
2
SinglePionTiming_2p2_4GeV
2
SinglePionTiming_2p2_4GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_50GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_6GeV
SinglePionTiming_2p2_8GeV
SinglePionTiming_2p2_8GeV
SinglePionTiming_2p2_8GeV
{100: [{'avg': 0.012548278205329902, 'hgcal': 0.014730072448355485, 'mtd': 0.024378520615900467}, {'avg': 8.413521607480336e-05, 'hgcal': 0.00010322132675843576, 'mtd': 0.00022005667347667503}], 10: [{'avg': 0.01693444382725754, 'hgcal': 0.021761114150342452, 'mtd': 0

Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_100GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_10GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_15GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_2GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file HSinglePionTiming_2p2_30GeV.pdf has been created
Info in <TCanvas::Print>: pdf file PlotAssoc/HSinglePionTiming_2p2_4GeV.pdf has been created
Info in <TCanvas:

# Multiplot 2p2

In [9]:


file = open('data_assoc/res2p2.pickle', 'rb')
dict_resolutions = pickle.load(file)
file.close()



g_dict2p2={"avg":ROOT.TGraphErrors(),
        "hgcal":ROOT.TGraphErrors(),
        "mtd":ROOT.TGraphErrors()}

print(dict_resolutions)
print(dict_resolutions[2])
counter=0
for enkey in dict_resolutions.keys():
    for typekey in dict_resolutions[enkey][0]:
        g_dict2p2[typekey].AddPoint(enkey*math.cosh(1.9),dict_resolutions[enkey][0][typekey])#,0.,dict_resolutions[enkey][1][typekey])
        g_dict2p2[typekey].SetPointError(counter,0,dict_resolutions[enkey][1][typekey])
    counter=counter+1
cmg2p2=ROOT.TCanvas()
cmg2p2.cd()
mg2p2=ROOT.TMultiGraph()


g_dict2p2["avg"].SetLineColor(ROOT.kMagenta+2)
g_dict2p2["avg"].SetMarkerColor(ROOT.kMagenta+2)
g_dict2p2["avg"].SetMarkerStyle(22)
g_dict2p2["avg"].SetTitle("Combination")

g_dict2p2["hgcal"].SetLineColor(ROOT.kCyan+2)
g_dict2p2["hgcal"].SetMarkerColor(ROOT.kCyan+2)
g_dict2p2["hgcal"].SetMarkerStyle(20)
g_dict2p2["hgcal"].SetTitle("HGCal time")


g_dict2p2["mtd"].SetLineColor(ROOT.kSpring+4)
g_dict2p2["mtd"].SetMarkerColor(ROOT.kSpring+4)
g_dict2p2["mtd"].SetMarkerStyle(34)
g_dict2p2["mtd"].SetTitle("MTD time")
#[g_dict[x].SetTitle(str(x)) for x in g_dict.keys()]

#[g_dict[x].SetMarkerStyle(20) for x in g_dict.keys()]


f_res_E.SetParameters(1.,1.,0.016)
f_res_E.SetParNames("S","N","C")


ROOT.gStyle.SetOptFit(True)

g_dict2p2["hgcal"].Fit(f_res_E,"r")

[mg2p2.Add(g_dict2p2[x]) for x in g_dict2p2.keys()]

mg2p2.SetTitle("Time resolution;E [GeV]; Resolution [ns]")
mg2p2.Draw("AP")

#cmg.BuildLegend()
legend=ROOT.TLegend(0.2,0.8,0.4,0.7)

legend.AddEntry(g_dict["avg"])
legend.AddEntry(g_dict["mtd"])
legend.AddEntry(g_dict["hgcal"])
legend.Draw()

text = ROOT.TLatex()
text.SetNDC(1)  # Imposta le coordinate in termini di frazione del canvas (0-1)
text.SetTextSize(0.04)  # Imposta la dimensione del testo
text.SetTextAlign(31)  # Allinea il testo a destra rispetto alla posizione specificata
text.DrawLatex(0.9, 0.92, "#eta=2.2")  # (x, y, testo) posizione nell'angolo in alto a destra
#mg.SetMinimum(0.)
#mg.SetMaximum(0.035)



mg2p2.SetMinimum(0.)
mg2p2.SetMaximum(0.040)

cmg2p2.Draw()

cmg2p2.SaveAs("PlotAssoc/Resolution_2p2_E_fit.pdf")



{100: [{'avg': 0.012548278205329902, 'hgcal': 0.014730072448355485, 'mtd': 0.024378520615900467}, {'avg': 8.413521607480336e-05, 'hgcal': 0.00010322132675843576, 'mtd': 0.00022005667347667503}], 10: [{'avg': 0.01693444382725754, 'hgcal': 0.021761114150342452, 'mtd': 0.02418783699286592}, {'avg': 0.0001316352179520598, 'hgcal': 0.00020279700959225419, 'mtd': 0.00022541407456867492}], 15: [{'avg': 0.015333711242872894, 'hgcal': 0.019334701236405898, 'mtd': 0.0245225027046509}, {'avg': 0.00011230795606268568, 'hgcal': 0.000158746635249872, 'mtd': 0.00022412635981730497}], 2: [{'avg': 0.021754758570414617, 'hgcal': 0.03596682832810902, 'mtd': 0.022541299877572273}, {'avg': 0.0003145400333864101, 'hgcal': 0.001525716365184281, 'mtd': 0.0003313140225542935}], 30: [{'avg': 0.013837082103279039, 'hgcal': 0.016928791255268395, 'mtd': 0.02482238455904847}, {'avg': 9.427523859385632e-05, 'hgcal': 0.00012309266807128725, 'mtd': 0.0002214876166602812}], 4: [{'avg': 0.020919108901877915, 'hgcal': 0.

Info in <TCanvas::Print>: pdf file PlotAssoc/Resolution_2p2_E_fit.pdf has been created


 # HGCal comparison

In [10]:


cmg1=ROOT.TCanvas()
cmg1.cd()
mg1=ROOT.TMultiGraph()

g_dict["hgcal"].SetTitle("HGCal, #eta=1.9")
g_dict["avg"].SetTitle("Combined, #eta=1.9")

mg1.Add(g_dict["hgcal"])
mg1.Add(g_dict["avg"])

g_dict2p2["hgcal"].SetLineColor(ROOT.kBlue+2)
g_dict2p2["hgcal"].SetMarkerColor(ROOT.kBlue+2)
g_dict2p2["hgcal"].SetTitle("HGCal, #eta=2.2")

g_dict2p2["avg"].SetLineColor(ROOT.kOrange+10)
g_dict2p2["avg"].SetMarkerColor(ROOT.kOrange+10)
g_dict2p2["avg"].SetTitle("Combined, #eta=2.2")




mg1.Add(g_dict2p2["hgcal"])
mg1.Add(g_dict2p2["avg"])

legendcmp=ROOT.TLegend(0.2,0.8,0.4,0.65)
legendcmp.AddEntry(g_dict["avg"])
legendcmp.AddEntry(g_dict["hgcal"])
legendcmp.AddEntry(g_dict2p2["avg"])
legendcmp.AddEntry(g_dict2p2["hgcal"])
mg1.SetTitle("Time resolution;E [GeV]; Resolution [ns]")

mg1.SetMinimum(0.)
mg1.SetMaximum(0.040)
mg1.Draw("AP")

legendcmp.Draw()


#cmg1.Draw()

cmg1.SaveAs("PlotAssoc/Comb_1p9_2p2_.pdf")

Info in <TCanvas::Print>: pdf file PlotAssoc/Comb_1p9_2p2_.pdf has been created
